In [1]:
from datetime import datetime
import numpy as np
import itertools
import nltk
import os
import operator
import sys

In [2]:
#Do this only the first time
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\anshr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\anshr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\anshr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\anshr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\anshr\AppData\Roaming\nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\anshr\AppData\R

True

In [3]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
corpora_dir = "C:/Users/anshr/AppData/Roaming/nltk_data/corpora/state_union"

In [4]:
print("Reading Data!!! OMG SOOO OFFICIALLL!! AAHHH\n\n\n")
file_list = []
for root,_,files in os.walk(corpora_dir):
    for filename in files:
        file_list.append(os.path.join(root,filename))

sentences = []
for files in file_list:
    with open(files,'r') as fin:
        try:
            str_form = fin.read().replace('\n','')
            sentences.extend(nltk.sent_tokenize(str_form))
        except:
            pass
print(sentences[:5])

Reading Data!!! OMG SOOO OFFICIALLL!! AAHHH



["PRESIDENT HARRY S. TRUMAN'S ADDRESS BEFORE A JOINT SESSION OF THE CONGRESS April 16, 1945Mr.", 'Speaker, Mr. President, Members of the Congress:It is with a heavy heart that I stand before you, my friends and colleagues, in the Congress of the United States.Only yesterday, we laid to rest the mortal remains of our beloved President, Franklin Delano Roosevelt.', 'At a time like this, words are inadequate.', 'The most eloquent tribute would be a reverent silence.Yet, in this decisive hour, when world events are moving so rapidly, our silence might be misunderstood and might give comfort to our enemies.In His infinite wisdom, Almighty God has seen fit to take from us a great man who loved, and was beloved by, all humanity.No man could possibly fill the tremendous void left by the passing of that noble soul.', 'No words can ease the aching hearts of untold millions of every race, creed and color.']


In [5]:
# Sentence delimiters
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

# Word fequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found ", len(word_freq.items()), " unique words...")

Found  19003  unique words...


In [6]:
print(len(sentences))
print(tokenized_sentences[:5])
print(word_freq)

13667
[['PRESIDENT', 'HARRY', 'S.', 'TRUMAN', "'S", 'ADDRESS', 'BEFORE', 'A', 'JOINT', 'SESSION', 'OF', 'THE', 'CONGRESS', 'April', '16', ',', '1945Mr', '.'], ['Speaker', ',', 'Mr.', 'President', ',', 'Members', 'of', 'the', 'Congress', ':', 'It', 'is', 'with', 'a', 'heavy', 'heart', 'that', 'I', 'stand', 'before', 'you', ',', 'my', 'friends', 'and', 'colleagues', ',', 'in', 'the', 'Congress', 'of', 'the', 'United', 'States.Only', 'yesterday', ',', 'we', 'laid', 'to', 'rest', 'the', 'mortal', 'remains', 'of', 'our', 'beloved', 'President', ',', 'Franklin', 'Delano', 'Roosevelt', '.'], ['At', 'a', 'time', 'like', 'this', ',', 'words', 'are', 'inadequate', '.'], ['The', 'most', 'eloquent', 'tribute', 'would', 'be', 'a', 'reverent', 'silence.Yet', ',', 'in', 'this', 'decisive', 'hour', ',', 'when', 'world', 'events', 'are', 'moving', 'so', 'rapidly', ',', 'our', 'silence', 'might', 'be', 'misunderstood', 'and', 'might', 'give', 'comfort', 'to', 'our', 'enemies.In', 'His', 'infinite', 'wis

In [7]:
# We limit the total words converted to tokens by giving priority based on frequency
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
print("Vocabulary size = ",vocabulary_size)
print("The least used word is \"{}\" and it appeared {} times".format(vocab[-1][0],vocab[-1][1]))
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

Vocabulary size =  8000
The least used word is "discriminating" and it appeared 2 times


In [8]:
# create training data
# x represents every word except for the last one, y represents every next word.
x_train = np.asarray([[word_to_index[w] for w in sentence[:-1]] for sentence in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sentence[1:]] for sentence in tokenized_sentences])


In [9]:
x_example, y_example = x_train[0], y_train[0]
print("The 10th input is {} and corresponding target output is {}".format(x_example, y_example))
print("In human terms, it looks something like: ")
print(list(zip([index_to_word[x] for x in x_example],[index_to_word[y] for y in y_example])))

The 10th input is [722, 3562, 3025, 3563, 822, 1113, 1385, 241, 1114, 1115, 437, 246, 757, 1992, 1720, 1, 7999] and corresponding target output is [3562, 3025, 3563, 822, 1113, 1385, 241, 1114, 1115, 437, 246, 757, 1992, 1720, 1, 7999, 2]
In human terms, it looks something like: 
[('PRESIDENT', 'HARRY'), ('HARRY', 'S.'), ('S.', 'TRUMAN'), ('TRUMAN', "'S"), ("'S", 'ADDRESS'), ('ADDRESS', 'BEFORE'), ('BEFORE', 'A'), ('A', 'JOINT'), ('JOINT', 'SESSION'), ('SESSION', 'OF'), ('OF', 'THE'), ('THE', 'CONGRESS'), ('CONGRESS', 'April'), ('April', '16'), ('16', ','), (',', 'UNKNOWN_TOKEN'), ('UNKNOWN_TOKEN', '.')]


In [10]:
def softmax(x):
    t = (np.exp(x)/np.sum(np.exp(x), axis = 0))
    print(t.shape)
    return t

In [11]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim = 50, bptt_truncate = 4):
        
        # Instance variable
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # Randomly initialize network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, word_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
    
    def forward_prop(self, x):
        time_steps = len(x)
        # we save all time steps during forward propagation
        states = np.zeros((time_steps+1, self.hidden_dim))
        # +1 for the first state we have initialised to 0
        states[-1] = np.zeros(self.hidden_dim)
        # we also save the outputs at each time step
        outs = np.zeros((time_steps,self.word_dim))
        # print(outs.shape)
        print(states.shape)
        for t in np.arange(time_steps):
            # indexing U by x[t] is the same as multiplying U with One Hot vector
            states[t] = np.tanh(self.U[:,x[t]] + self.W.dot(states[t-1]))
            s_in = self.V.dot(states[t])
            print(s_in.shape)
            outs[t] = softmax(s_in)
        return [outs,states]
    
    def predict(self, x):
        # Perform forward propagation and return index of highest score
        outs, states = self.forward_prop(x)
        return np.argmax(outs, axis = 1)
    
    def calculate_total_loss(self, x, y):
        L = 0
        for i in np.arange(len(y)):
            o,s = self.forward_prop(x[i])
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            L += -1 * np.sum(np.log(correct_word_predictions))
    
    def calculate_loss(self, x, y):
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x,y)/N
    
    def sgd_step(self, x, y, learning_rate):
        # calculate the gradients
        dldu, dldv, dldw = self,bptt(x,y)
        self.U -= learning_rate * dldu
        self.V -= learning_rate * dldv
        self.W -= learning_rate * dldw
    
    def bptt(self, x, y):
        T = len(y)
        o,s = self.forward_prop(x)
        dldu = np.zeros(self.U.shape)
        dldv = np.zeros(self.V.shape)
        dldw = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        for t in np.arange(T)[::-1]:
            dldv += np.outer(delta_o[t], s[t].T)
            delta_t = self.V.T.dot(delta_o[t]) * (1-(s[t] ** 2))
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                dldw += np.outer(delta_t, s[bptt_step-1])
                dldu[:, x[bptt_step]] += delta_t
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dldu, dldv, dldw]
    
    def gradient_check(self, x, y, h = 0.001, error_threshold = 0.01):
        bptt_gradients = model.bptt(x,y)
        model_parameters = ["U", "V", "W"]
        for pidx, pname in enumerate(model_parameters):
            parameter = operator.attrgetter(pname)(self)
            print("Performing gradient check for parameter {} with size {}".format(pname, np.prod(parameter.shape)))
            it = np.idter(parameter, flags = ['multi_index'], op_flags = ['readwrite'])
            while not it.finished:
                ix = it.multiindex
                original_value = parameter[ix]
                parameter[ix] = original_value + h
                gradplus = model.calculate_total_loss([x],[y])
                parameter[ix] =original_value - h
                gradminus = model.calculate_total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                parameter[ix] = original_value
                backprop_gradient = bptt_gradients[pidx][ix]
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                if relative_error > error_threshold:
                    print("Gradient Check ERROR: parameters {} ix = {}".format(pname, ix))
                    print("+h loss: ", gradplus)
                    print("-h loss: ", gradminus)
                    print("Estimated_gradient: ", estimated_gradient)
                    print("Backprop gradient: ", backprop_gradient)
                    print("Relative Error: ", relative_error)
                    return
                it.iternext()
            print("Gradient check passed for parameter: ", pname)

In [12]:
np.random.seed(10)
model = RNN(vocabulary_size)
o, s = model.forward_prop(x_train[10])
print(o.shape)
print(o)

(10, 50)


ValueError: shapes (50,8000) and (50,) not aligned: 8000 (dim 1) != 50 (dim 0)